## Setting up Pauli spin blockade

### Introduction
In this tutorial we will demonstrate how to set up spin-to-charge conversion using Pauli spin blockade.

In [ ]:
import Pyro4
from qick import QickConfig
from spinqick.settings import file_settings
from spinqick.models import spam_models, full_experiment_model, dcs_model
from spinqick.helper_functions import file_manager, hardware_manager
from spinqick.experiments import psb_setup

In [ ]:
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION = 4

ns_host = "192.168.2.99"  # make sure this matches your board's ip address!
ns_port = 8888
proxy_name = "myqick"

ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)

### SPAM Config
First, lets revisit the experiment config file that we set up in `00_getting_started.ipynb`. As we set up SPAM for our device, we are going to be editing the `DefaultSpam` model for our qubit.  The parameters in this model are used by the functions in `spinqick.core.readout_v2.py`. If you want to write your own code to run on qick, you can import the `readout_v2` module in `core` and use these premade functions. If you use a different type of readout or spin-to-charge conversion, add it to spinQICK, we would love to help!

#### SPAM Sequence

<p align="center">
<img src="demo_figures/SPAM_sequence.png" alt="SPAM Sequence" width="75%"/>
<figcaption><b>Figure 1. SPAM Sequence:</b>The above state-preparation and measurement sequence shown in (a) enumerates the sequence steps and their corresponding function in the PsbSetup class. Each step can act as a ramp defined with voltage duration pairs. (b) shows the same sequence in voltage space, showing how the entry and exit from the (1,1) and (2,0) charge cells from the measurement window (point C) would appear. For a reference measurement, the <i>exit_11</i> command would be omitted, and the sequence would remain at point C.</figcaption>
</p>

Spin-to-charge routines in spinqick are broken into SPAM 'steps', and each step may either be a single coordinate in voltage space, or a ramp between two points. These are defined by the `SpamRamp` and `SpamPulse` models. For each spam step, the user can define a set of gates and voltages to play on each gate, as well as a duration (in microseconds). The `DefaultSpam` model is broken into six steps, shown above in Fig 1. a, - `flush`, `entry_20`, `meas`, `exit_11`, `idle`, and `entry_11`. SpinQICK will play the sequence `flush` -> `entry_20` ->  `exit_11` -> `idle` before coherent manipulations on the qubit (at the idle point E), and `entry_11` -> `meas` after. This is if the `reference` argument in `ReadoutParams` is `True`, spinQICK will also execute a reference measurement (`flush` -> `entry_20` ->  `meas`) first.

Here is an example of how to set up an experiment config file which includes SPAM settings:

In [ ]:
# basic DCS readout parameters
dcs_example = dcs_model.DcsConfigParams(
    adc_trig_offset=0.0,
    dds_freq=1,
    readout_freq=1,
    length=10,
    readout_length=10,
    pulse_gain_readout=0.9,
    slack_delay=0.0,
)

# SPAM step definitions
flush = spam_models.SpamStep(
    duration=10,
    gate_list={
        "P1": spam_models.SpamPulse(voltage=-0.01),
        "P2": spam_models.SpamPulse(voltage=-0.05),
    },
)
measure = spam_models.SpamStep(
    duration=10,
    gate_list={
        "P1": spam_models.SpamPulse(voltage=0),
        "P2": spam_models.SpamPulse(voltage=0),
    },
)
idle = spam_models.SpamStep(
    duration=10,
    gate_list={
        "P1": spam_models.SpamPulse(voltage=0),
        "P2": spam_models.SpamPulse(voltage=0.05),
    },
)  # set idle time which is long enough for a singlet to dephase
entry_20 = spam_models.SpamStep(
    duration=0.1,
    gate_list={
        "P1": spam_models.SpamRamp(voltage=0.01, voltage_2=0),
        "P2": spam_models.SpamRamp(voltage=-0.01, voltage_2=0),
    },
)
exit_11 = spam_models.SpamStep(
    duration=0.1,
    gate_list={
        "P1": spam_models.SpamRamp(voltage=0, voltage_2=-0.01),
        "P2": spam_models.SpamRamp(voltage=0, voltage_2=0.01),
    },
)
entry_11 = spam_models.SpamStep(
    duration=0.1,
    gate_list={
        "P1": spam_models.SpamRamp(voltage=-0.01, voltage_2=0),
        "P2": spam_models.SpamRamp(voltage=0.01, voltage_2=0),
    },
)

psb_cfg = spam_models.DefaultSpam(
    flush=flush,
    entry_20=entry_20,
    meas=measure,
    entry_11=entry_11,
    exit_11=exit_11,
    idle=idle,
)
ro_cfg = full_experiment_model.ReadoutParams(
    psb_cfg=psb_cfg,  # DefaultSpam config
    measure_dot="M1",  # which dot to perform readout with
    reference=True,  # whether to perform a reference measurement
    thresh=False,  # whether to threshold the data (to 1 and 0)
    threshold=10,  # threshold value, if thresh is true
)

We assign this readout to a qubit, and add it to the qubit_configs dict in the Experiment config. Here I name the qubit "q1". You can add qubit definitions to your `qubit_configs` dict as you see fit. Experiments will take the qubit name as an argument once defined.

In [ ]:
q1 = full_experiment_model.QubitParams(ro_cfg=ro_cfg, qubit_params=None)
full = full_experiment_model.ExperimentConfig(
    m1_readout=dcs_example, m2_readout=dcs_example, qubit_configs={"q1": q1}
)

# if you want to save this experiment config and use it yourself:
full_cfg = file_settings.dot_experiment_config
file_manager.save_config_json(full, full_cfg)

### Setting up Pauli spin blockade (PSB)

We start by initializing the `PsbSetup` class located in the `psb_setup` sub-module. This class contains methods for performing 2D sweeps of idle, measure and flush points.  We can perform the sweeps and edit our json file to tweak the coordinates and durations of these points.

In [ ]:
vdc = hardware_manager.DummyDCSource()
psb = psb_setup.PsbSetup(soccfg, soc, voltage_source=vdc, save_data=False, plot=False)

# Specify which qubit to setup readout on
psb.qubit = "q1"

We can now run sweeps of the gates to find the measurement and flush windows using the `meas_window_scan` and `flush_window_scan` methods in `PsbSetup` respectively

In [ ]:
meas = psb.meas_window_scan(
    p_gates=("P1", "P2"),
    p_range=((-0.01, 0.05, 100), (-0.04, 0.005, 100)),
    step_time=0.02,
    point_avgs=5,
    full_avgs=2,
)

![meas](demo_data/1754589861_meas_scan.png)

In [ ]:
flush = psb.flush_window_scan(
    p_gates=("P1", "P2"), p_range=((-0.01, 0.05, 100), (-0.04, 0.005, 100)), point_avgs=5, full_avgs=2
)

![flush](demo_data/1754589529_flush_scan.png)

Once appropriate values are determined for initialization and readout, and placed in the `psb_config`, one can determine the readout fidelity of singlet and triplet states using the `meashist` function.

In [ ]:
mh = psb.meashist(10000, use_gmm=True)

![meashist](demo_data/1754589287_meas_hist.png)